<a href="https://colab.research.google.com/github/Hagar-Usama/Chest-X-ray-diagnosis/blob/basic-models/Final_Chest_X_ray_Diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TODO:
* [x] FCNN1 
* [x] FCNN 2
* [x] CNN 1
* [x] CNN 2
* [x] Data Augmentation (bonus) [double-check]
* [x] ResNet  (pretrained = true & false)
* [x] DenseNet
* [x] Inception v3 (bonus)
* [x] Compare all model in a table (use Tabulate)

# Import packages

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
import cv2
from google.colab import drive
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
import torch
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split
from google.colab import files
from torchvision.models import resnet18, resnet50
from statistics import mode

In [ ]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

# Load Data

In [ ]:
drive.mount('/content/drive') #, force_remount=True)
path = '/content/drive/My Drive/COVID-19RadiographyDatabase'
labels = []
data = []
dim = (256, 256)   

for directory in os.listdir(path):
  for file in os.listdir(path + '/' + directory):
    labels.append(directory)
    imagedata = cv2.imread(path + "/" + directory + "/" + file)
    resized = cv2.resize(imagedata, dim, interpolation = cv2.INTER_AREA)
    data.append(resized)
    #data.append(imagedata)
print(len(data))
print(len(labels))

Mounted at /content/drive


KeyboardInterrupt: ignored

**Pickle** File (less storage, faster retreival)
* Alteranative to the previous step

In [ ]:
## Credit
# https://medium.com/better-programming/load-fast-load-big-with-compressed-pickles-5f311584507e
import pickle
import bz2
import os
import cv2
import _pickle as cPickle

def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        cPickle.dump(data, f)

# Load any compressed pickle file
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## run iff a pickle file is used
from pathlib import Path
dataset_path = Path("/content/drive/My Drive/covid-19")
!ls "/content/drive/My Drive/covid-19"
labels = decompress_pickle(dataset_path.joinpath("covid-compressed-label.pbz2"))
print(len(labels))
data = decompress_pickle(dataset_path.joinpath("covid-compressed-image.pbz2"))
print(len(data))
#print(type(data[0]))
# mind: 3865 >> missing data (36 image)

COVID-19		     covid-compressed-label.pbz2  many_fcnn_2_mod
covid-compressed-image.pbz2  covid-compressed-size.pbz2   many_fcnn.pbz2
3829
3829


# Change Text Labels to Numerical

In [ ]:
le = LabelEncoder()
labels = le.fit_transform(labels)
print(labels)
print(data[0].shape)

[2 2 2 ... 1 1 1]
(256, 256, 3)


# Split Dataset

In [ ]:
x_train, x_test_cv, y_train, y_test_cv = train_test_split(data, labels, test_size=0.4, random_state=1, stratify=labels)
x_cv,    x_test,    y_cv,    y_test    = train_test_split(x_test_cv, y_test_cv, test_size=0.5, random_state=1, stratify=y_test_cv)

## Data Augmentation

In [ ]:
from torchvision import transforms, utils
train_trans =  transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

valid_trans = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

# Data Loader

In [ ]:
class myDataset(Dataset):
  def __init__(self, array, label):
    self.array = array
    self.label = label
    # for data augmentation
    # self.transform = transform
  
  def __getitem__(self, index):
    # image = cv2.imread(self.array[index], 0)
    # # # image = image.reshape(1,28,28)
    image = self.array[index]
    #image = self.transform(image)
    return torch.tensor(image).float(), torch.tensor(int(self.label[index]))
    # return(self.array[index], self.label[index])
  
  def __len__(self):
      return len(self.array)


In [ ]:
customDataset = myDataset(x_train, y_train)
train_dataloader = DataLoader(customDataset, batch_size=4, shuffle=True, num_workers=0)

customDataset = myDataset(x_cv, y_cv)
cv_dataloader = DataLoader(customDataset, batch_size=4, shuffle=True, num_workers=0)

customDataset = myDataset(x_test, y_test)
test_dataloader = DataLoader(customDataset, batch_size=4, shuffle=True, num_workers=0)

# Neural Networks Declaration

## CNN 1 (few layers)

In [ ]:
# https://towardsdatascience.com/9-tips-for-training-lightning-fast-neural-networks-in-pytorch-8e63a502f565
# really bad idea. Stops all the GPUs until they all catch up
torch.cuda.empty_cache()
class ConvNN(nn.Module):
  def __init__(self):
    super(ConvNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
    self.dropout1 = nn.Dropout(0.25)
    self.dropout2 = nn.Dropout(0.5)
    self.fc1 = nn.Linear(1016064, 128) 
    self.fc2 = nn.Linear(128, 3)

  def forward(self, x):
    x = self.conv1(x.permute(0, 3, 1, 2))
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x,2)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output

# conv_nn = ConvNN()
# fully_conn_nn = FullyConnectedNN()
# resnet = ResNet50()

## CNN 2 (manny layers)

In [ ]:
torch.cuda.empty_cache()
class ConvNN2(nn.Module):
  def __init__(self):
    super(ConvNN2, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1)
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1)
    self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1)
    # self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
    # print(self.conv4.weight.shape)
    
    # print(self.conv4.weight)
    self.dropout1 = nn.Dropout(0.25)
    self.dropout2 = nn.Dropout(0.5)

    self.fc1 = nn.Linear(500000, 128) 
    self.fc2 = nn.Linear(128, 3)

  def forward(self, x):
    x = self.conv1(x.permute(0, 3, 1, 2))
    x = F.relu(x)

    x = self.conv2(x)
    x = F.relu(x)

    x = self.conv3(x)
    
    x = F.relu(x)

    # x = self.conv4(x)
   

    # x = F.relu(x)

    x = F.max_pool2d(x,2)
    x = self.dropout1(x)

    x = torch.flatten(x, 1)
    # print(x.size())
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output

# temp = ConvNN2()

## FCNN1

In [ ]:
input_size = 256*256*3 #65536
output_size = 3

class FullyConnectedNN(nn.Module):
    
    def __init__(self):
        super(FullyConnectedNN, self).__init__()
        self.l1 = nn.Linear(input_size, 500)
        self.l3 = nn.Linear(500, 100 )
        self.l6 = nn.Linear(100, output_size)
        self.dropout2 = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.flatten(x, 1)
        
        x = self.l1(x)
        x = F.relu(x)
        x = self.dropout2(x)

        x = self.l3(x)
        x = F.relu(x)
        x = self.dropout2(x)

        x = self.l6(x)
        return x

## ResNet50

In [ ]:
class ResNet50(nn.Module):
  def __init__(self):
    super(ResNet50, self).__init__()
    self.layer1 = resnet50(pretrained=False)
    self.layer2 = nn.Softmax(dim=1)
  
  def forward(self, x):
    x = self.layer1(x.permute(0, 3, 1, 2))
    x = self.layer2(x)
    return x


## FCNN 2

In [ ]:
# hyperparameters
input_size = 256*256*3 #65536
output_size = 3
hidden_size = 1000
hidden_size0 = 500
hidden_size1 = 128
hidden_size2 = 84
hidden_size3 = 10

# many layers FCNN model
class FullyConnectedNN2(nn.Module):
    
    def __init__(self):
        super(FullyConnectedNN2, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l3 = nn.Linear(hidden_size, hidden_size0 )
        self.l4 = nn.Linear(hidden_size0, hidden_size1)
        self.l5 = nn.Linear(hidden_size1, hidden_size2)
        self.l6 = nn.Linear(hidden_size2, output_size)
        self.dropout = nn.Dropout(0.5)
        self.tanh = nn.Tanh()
        self.l7 = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.l1(x)
        x = self.relu(x)
        x = self.l3(x)
        x = self.relu(x)
        x = self.l4(x)
        x = self.tanh(x)
        x = self.l5(x)
        x = self.dropout(x)
        x = self.l6(x)
        x = self.tanh(x)
        x = self.l7(x)
        return x

## Initialize Weights

In [ ]:
def weights_init_uniform_rule(m):
        classname = m.__class__.__name__
        # for every Linear layer in a model..
        if classname.find('Linear') != -1:
            # get the number of the inputs
            n = m.in_features
            y = 1.0/np.sqrt(n)
            m.weight.data.uniform_(-y, y)
            m.bias.data.fill_(0)

# Get Metrics

In [ ]:
def get_metrics(pred, labels):
  # pred = torch.exp(torch.tensor(pred)) # get probabilities back
  pred = torch.tensor(pred)
  _, pred = torch.max(pred, 1)
  # _, pred = torch.max(torch.tensor(pred), 1)
  # print(pred)
  f1 = f1_score(labels, pred.cpu(), average='weighted')
  r = recall_score(labels, pred.cpu(), average='weighted')
  p = precision_score(labels, pred.cpu(), average='weighted')
  a = accuracy_score(labels, pred.cpu())
  print("f1 score: ", f1)
  print("recall: ", r)
  print("precision: ", p)
  print("accuracy: ", a)


# Models Training and Testing

In [ ]:
def run_model(model, dataloader, optimizer, scheduler, train = True):
  if train:
    model.train()
  else:
    model.eval()

  pred = []
  labels = []
  loss = nn.CrossEntropyLoss()
  total_loss = 0

  for (data, label) in dataloader:  
      # zero the parameter gradients
      optimizer.zero_grad()
      # forward + backward + optimize
      with torch.set_grad_enabled(train == True):
        output = model(data)
        loss_ = loss(output, label)

      total_loss += loss_.item()

      if train:
        loss_.backward()
        optimizer.step()

      pred += output.tolist()
      labels += label.tolist()
        

  return labels,pred,total_loss/len(dataloader)

In [ ]:
def train_model(model, optimizer, scheduler, epoch):

  for e in range(epoch):
    print("\n**********epoch: ", e, "\n")
    labels, pred, loss = run_model(model, train_dataloader, optimizer, scheduler, train=True)
    print("\ntrain:\n") 
    print("loss: ", loss)
    get_metrics(pred, labels)

    with torch.no_grad():
      labels, pred, loss = run_model(model, cv_dataloader, optimizer, scheduler, train=False)
      scheduler.step(loss)
      print("\nvalidation:\n")
      print("loss: ", loss)
      get_metrics(pred, labels)

In [ ]:
def test_model(model, optimizer, scheduler):
  with torch.no_grad():
    labels, pred, loss = run_model(model, test_dataloader, optimizer, scheduler, train=False)
    print("test: ")
    print("loss: ", loss)
    print(pred)
    print(torch.exp(torch.tensor(pred)))
    print(labels)

    get_metrics(pred, labels)

# ConvNN Training

In [ ]:
convNN_model = ConvNN()
torch.set_default_tensor_type('torch.cuda.FloatTensor')
convNN_model.cuda()
optimizer = torch.optim.Adam(convNN_model.parameters(), 0.0000001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=0.0001) 
epoch = 15
train_model(convNN_model, optimizer, scheduler, epoch)


**********epoch:  0 


train:

loss:  1.56740935869191
f1 score:  0.5754908500740619
recall:  0.5777100565955594
precision:  0.5739427176545164
accuracy:  0.5777100565955594

validation:

loss:  0.608917397679761
f1 score:  0.7446694905457457
recall:  0.7558746736292428
precision:  0.8015017656914472
accuracy:  0.7558746736292428

**********epoch:  1 


train:

loss:  0.7114696058035707
f1 score:  0.6774198106275648
recall:  0.6804527644754027
precision:  0.6760392612571887
accuracy:  0.6804527644754027

validation:

loss:  0.519385039845171
f1 score:  0.8186737226919126
recall:  0.8198433420365535
precision:  0.8331706124740287
accuracy:  0.8198433420365535

**********epoch:  2 


train:

loss:  0.6170627337726562
f1 score:  0.7316062331654064
recall:  0.7331301697866782
precision:  0.7309120929837853
accuracy:  0.7331301697866782

validation:

loss:  0.43302673877527315
f1 score:  0.8481305472668015
recall:  0.8485639686684073
precision:  0.8509206285924394
accuracy:  0.848563968668

# ConvNN Testing

In [ ]:
test_model(convNN_model, optimizer, scheduler)

test: 
loss:  0.2628325190792869
[[-0.002975443610921502, -6.327452182769775, -6.73854398727417], [-6.216653823852539, -0.39576029777526855, -1.1244347095489502], [-0.029449280351400375, -4.161752700805664, -4.309554100036621], [-8.49248218536377, -0.07039942592382431, -2.6915833950042725], [-0.30678218603134155, -3.8276150226593018, -1.4170515537261963], [-0.0005298641044646502, -8.029641151428223, -8.49710464477539], [-3.662538528442383, -4.459349632263184, -0.037948109209537506], [-6.371342182159424, -1.5706210136413574, -0.23524898290634155], [-0.09038940072059631, -3.9734151363372803, -2.6939165592193604], [-7.028767108917236, -2.3325676918029785, -0.1030656099319458], [-0.005440903827548027, -7.263734817504883, -5.354755878448486], [-6.240358352661133, -0.8339428901672363, -0.5732021331787109], [-10.392414093017578, -2.034060478210449, -0.14022113382816315], [-3.073364019393921, -3.0593678951263428, -0.09781425446271896], [-7.201408386230469, -0.08779143542051315, -2.485275268554

# ConvNN 2 Training

In [ ]:
convNN2_model = ConvNN2()
torch.set_default_tensor_type('torch.cuda.FloatTensor')
convNN2_model.cuda()
optimizer = torch.optim.Adam(convNN2_model.parameters(), 0.0000001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=0.0001) 
epoch = 15
train_model(convNN2_model, optimizer, scheduler, epoch)


**********epoch:  0 


train:

loss:  1.170556653427041
f1 score:  0.5188722246295974
recall:  0.5193730953417501
precision:  0.5184603621112841
accuracy:  0.5193730953417501

validation:

loss:  0.6245144951778153
f1 score:  0.7406628418434253
recall:  0.7545691906005222
precision:  0.8035612645497942
accuracy:  0.7545691906005222

**********epoch:  1 


train:

loss:  0.7360932179767152
f1 score:  0.654181965638906
recall:  0.6547670875054419
precision:  0.6536478705499698
accuracy:  0.6547670875054419

validation:

loss:  0.561578401674827
f1 score:  0.7752177275464972
recall:  0.7819843342036553
precision:  0.8227594050772776
accuracy:  0.7819843342036553

**********epoch:  2 


train:

loss:  0.6446862002041029
f1 score:  0.6991277259206918
recall:  0.6987374836743578
precision:  0.6995410032559731
accuracy:  0.6987374836743578

validation:

loss:  0.49359505200603354
f1 score:  0.8395031288165168
recall:  0.839425587467363
precision:  0.8472272731188949
accuracy:  0.839425587467

# ConvNN 2 Testing

In [ ]:
test_model(convNN2_model, optimizer, scheduler)

test: 
loss:  0.2576743458582011
[[-0.0039704786613583565, -8.585112571716309, -5.5791544914245605], [-5.92579984664917, -2.499952554702759, -0.08856747299432755], [-0.08042802661657333, -4.948905944824219, -2.6565847396850586], [-5.94563627243042, -0.012924239039421082, -4.583034515380859], [-6.661315441131592, -4.134222030639648, -0.017445918172597885], [-7.765295505523682, -0.5065224766731262, -0.9238484501838684], [-4.294979572296143, -4.47038459777832, -0.025399569422006607], [-0.0007657456444576383, -7.810447692871094, -7.929582595825195], [-0.01269509270787239, -6.763330459594727, -4.468937873840332], [-6.822900295257568, -2.2014846801757812, -0.11847653985023499], [-5.157100677490234, -3.6770591735839844, -0.03154805302619934], [-0.029138097539544106, -6.547063827514648, -3.60147762298584], [-0.1763051301240921, -3.4143130779266357, -2.04996919631958], [-7.749880790710449, -0.7435449361801147, -0.6459895372390747], [-4.172796249389648, -0.42863720655441284, -1.0990283489227295]

# Fully Connected NN 1 Training 

In [ ]:
FCNN_model = FullyConnectedNN() 
torch.set_default_tensor_type('torch.cuda.FloatTensor')
FCNN_model.cuda()
optimizer = torch.optim.Adam(FCNN_model.parameters(), 0.0000001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=0.0001)
epoch = 20
train_model(FCNN_model, optimizer, scheduler, epoch)


**********epoch:  0 


train:

loss:  6.452208176853143
f1 score:  0.4918521411077783
recall:  0.4932520679146713
precision:  0.4937194262122208
accuracy:  0.4932520679146713

validation:

loss:  0.6812100075315944
f1 score:  0.652060566469318
recall:  0.6840731070496083
precision:  0.7335493126854835
accuracy:  0.6840731070496083

**********epoch:  1 


train:

loss:  1.782327611767951
f1 score:  0.5989141689295682
recall:  0.5999129299085764
precision:  0.5994964019819369
accuracy:  0.5999129299085764

validation:

loss:  0.5588674708607186
f1 score:  0.7127540145860984
recall:  0.7193211488250653
precision:  0.73146108054436
accuracy:  0.7193211488250653

**********epoch:  2 


train:

loss:  1.326358643334926
f1 score:  0.6322029185658544
recall:  0.6325642141924249
precision:  0.6324208306527617
accuracy:  0.6325642141924249

validation:

loss:  0.5005191373056732
f1 score:  0.7774084032691383
recall:  0.779373368146214
precision:  0.7896773469457048
accuracy:  0.779373368146214


# Fully Connected NN Testing

In [ ]:
test_model(FCNN_model, optimizer, scheduler)

test: 
loss:  0.3086287132731134
[[-1.0536218881607056, 0.42267727851867676, 1.762567162513733], [-2.4652631282806396, 1.8602792024612427, 5.023184776306152], [-1.6865485906600952, 5.545985221862793, -0.6817073822021484], [-1.3980319499969482, 0.30474773049354553, 2.338869571685791], [-2.43387770652771, 0.9230967164039612, 3.66454815864563], [2.599747896194458, -3.859297275543213, -3.7514231204986572], [5.307176113128662, -5.367149829864502, -5.219605445861816], [-0.747420608997345, 3.3500421047210693, -0.2506072521209717], [3.5325963497161865, -3.385493040084839, -5.867565155029297], [0.25953346490859985, -0.8671139478683472, -0.31101638078689575], [-0.5619250535964966, 1.6510854959487915, 0.6197570562362671], [-1.9082672595977783, 5.005702018737793, -0.30154871940612793], [-2.102691411972046, 1.0941202640533447, 4.0134077072143555], [-1.161272644996643, 2.3753745555877686, -0.3446914851665497], [-0.5402453541755676, 1.0172202587127686, 0.6858993172645569], [-2.020009756088257, 0.7751

# Fully Connected NN 2 Training

data input (x,256,256) I am using greyscale images

In [ ]:
def weights_init_uniform_rule(m):
        classname = m.__class__.__name__
        # for every Linear layer in a model..
        if classname.find('Linear') != -1:
            # get the number of the inputs
            n = m.in_features
            y = 1.0/np.sqrt(n)
            m.weight.data.uniform_(-y, y)
            m.bias.data.fill_(0)

In [ ]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
fully_connected_nn2 = FullyConnectedNN2()
model = fully_connected_nn2
model.apply(weights_init_uniform_rule)

FullyConnectedNN2(
  (l1): Linear(in_features=196608, out_features=1000, bias=True)
  (relu): ReLU()
  (l3): Linear(in_features=1000, out_features=500, bias=True)
  (l4): Linear(in_features=500, out_features=128, bias=True)
  (l5): Linear(in_features=128, out_features=84, bias=True)
  (l6): Linear(in_features=84, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (tanh): Tanh()
  (l7): Softmax(dim=1)
)

In [ ]:
epochs = 20
learning_rate = 0.0000001
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.5)
optimizer = torch.optim.Adadelta(model.parameters(), lr=0.0001, rho=0.9, eps=1e-06, weight_decay=0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=0.0001)


In [ ]:
for epoch in range(epochs):
  labels, pred, train_loss = run_model(model, train_dataloader, optimizer, scheduler, train=True)
  get_metrics(pred,labels)
  labels, pred, valid_loss = run_model(model, cv_dataloader, optimizer, scheduler, train=False)
  get_metrics(pred, labels)
  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))


f1 score:  0.8895553941652511
recall:  0.8894209838920331
precision:  0.8899889368861146
accuracy:  0.8894209838920331
f1 score:  0.8917349697196453
recall:  0.891644908616188
precision:  0.8919447845682879
accuracy:  0.891644908616188
Epoch: 0 	Training Loss: 0.867683 	Validation Loss: 0.852164
f1 score:  0.8932318810591946
recall:  0.8933391380060949
precision:  0.8932542566684046
accuracy:  0.8933391380060949
f1 score:  0.8817097278823354
recall:  0.8825065274151436
precision:  0.8827098905158198
accuracy:  0.8825065274151436
Epoch: 1 	Training Loss: 0.863621 	Validation Loss: 0.857336
f1 score:  0.8959534974174348
recall:  0.8959512407488028
precision:  0.8959808504192357
accuracy:  0.8959512407488028
f1 score:  0.8889431607754723
recall:  0.8890339425587467
precision:  0.8896319589000732
accuracy:  0.8890339425587467
Epoch: 2 	Training Loss: 0.858326 	Validation Loss: 0.848190
f1 score:  0.9028865562521246
recall:  0.9029168480626905
precision:  0.9028611285619601
accuracy:  0.902

# Fully Connected NN 2 Testing

In [ ]:
labels, pred, loss = run_model(model, test_dataloader, optimizer, scheduler, train=False)
print("test: ")
print("loss: ", loss)
print(pred)
print(torch.exp(torch.tensor(pred)))
print(labels)
get_metrics(pred, labels)

# pred = torch.exp(torch.tensor(pred))
# _, pred = torch.max(pred, 1)
# # _, pred = torch.max(torch.tensor(pred), 1)
# print(pred)
# f1 = f1_score(labels, pred.cpu(), average='weighted')
# r = recall_score(labels, pred.cpu(), average='weighted')
# p = precision_score(labels, pred.cpu(), average='weighted')
# print("f1 score: ", f1)
# print("recall: ", r)
# print("precision: ", p)



test: 
loss:  0.8074933458119631
[[0.13790839910507202, 0.7036963105201721, 0.15839530527591705], [0.13676989078521729, 0.1581399142742157, 0.7050902247428894], [0.7305358052253723, 0.13536979258060455, 0.13409434258937836], [0.13791339099407196, 0.7036883234977722, 0.15839827060699463], [0.1349443644285202, 0.15818192064762115, 0.7068737149238586], [0.7374676465988159, 0.12958908081054688, 0.13294324278831482], [0.13981489837169647, 0.15911434590816498, 0.7010707855224609], [0.12863919138908386, 0.5378192067146301, 0.333541601896286], [0.1382986307144165, 0.7014603614807129, 0.1602410078048706], [0.736765444278717, 0.1282004863023758, 0.13503406941890717], [0.7374655604362488, 0.12958340346813202, 0.13295109570026398], [0.13790152966976166, 0.7037364840507507, 0.1583620309829712], [0.14132097363471985, 0.16062182188034058, 0.698057234287262], [0.13538584113121033, 0.15945036709308624, 0.7051637768745422], [0.17571091651916504, 0.18711452186107635, 0.637174665927887], [0.13647104799747

# Improving Models
> [Link](https://colab.research.google.com/drive/1TIGtwZBQbuD5zhGne2xFA-ezkkUOu0ln?usp=sharing)